Problem 4: Cryptanalysis of a Many-Time Pad

In [9]:
from collections import Counter #used to create frequency counter function
import enchant #for spell checking function for automatically checking plaintext results
 
#Provided ciphertexts
c1 = """71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840"""
c2 = """71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3"""
c3 = """72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9"""
c4 = """67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2ffbd6a291571e40ba1afd3b4b1fe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd"""
c5 = """71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06"""
c6 = """6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b8940bb08e3f313cb23b3dfd8645cfcd80ff82489"""
c7 = """71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c900fb14db4b229ca74b6c4d96442e1c34df8288f5c3a450a527ecc7c82865b8e"""
c8 = """71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf5"""
c9 = """71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee448950efd5cabe500c120f2d9d26440ebc34de029811977430b01748276d79012955cc6a65aebb9054becda5c9278"""
c10 = """71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e257"""
c_target = """71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241da2eacff9cc8d5123fe5a897efbeda4974b"""


#create a list of the provided hex ciphertexts
original_ciphertexts = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]
#list for storing the ciphertexts as byte strings
encoded_ciphertests = []
#list for storing the resulting xored byte strings
xoredciphertexts_list= []
#dictionary of the same values to make it easier to see what two cipher texts are XORed and try crib XORing
xoredciphertexts_dict= {}
#list for storing the xored hex strings for frequency analysis
xoredciphertexts_hex_list= []
#dictionary of the same values make it easier to see what two cipher texts are XORed and try crib XORing
xoredciphertexts_hex_dict = {}
#list for storying results of crib dragging technique for frequency analysis
cribbedtexts = []
frequencycounts = []
#creates a dictionary to spell check plaintext strings after they have been XORed
d = enchant.Dict("en_US")
keyfragments = []

#encode the ciphertexts into an XORable format
for i in range(len(original_ciphertexts)):
    encoded_ciphertests.append(bytes.fromhex(original_ciphertexts[i]))

#function for XOR two of the bytestrings

def xor_strings(x,y):
    return bytes(a^b for a,b in zip(x,y))

#XOR the ciphertexts with each other, store the results in xoredciphertexts 
for i in range(len(encoded_ciphertests)):
    for j in range(i+1, len(encoded_ciphertests)):
        key = f"c{i+1} xor c{j+1}"
        xored_res = xor_strings(encoded_ciphertests[i],encoded_ciphertests[j])
        xoredciphertexts_dict[key] = xored_res
        xoredciphertexts_hex_dict[key] = xored_res.hex()
        xoredciphertexts_list.append(xored_res)
        xoredciphertexts_hex_list.append(xored_res)
        
    

#create a function to try xoring different cribs  
def crib_xor(crib, key):
    crib_res = []
    xoredciphertext = xoredciphertexts_dict[key]
    for i in range(0, len(xoredciphertext), len(crib)):
        chunk = xoredciphertext[i:i+len(crib)]
        if len(chunk) == len(crib):
            res = xor_strings(crib, chunk)
            crib_res.append(res)
            #if(d.check(res.hex()) == True):
            #    keyfragments.append()
            print(f"At index {i}: {res}")
    return(crib_res)
            
#allow manual input of which two ciphertexts to xor crib with  
#crib = input("Please enter a crib word: ").encode()
crib = "the".encode()
a,b = input("Please enter the numbers of the two ciphertexts to XOR separated by a space: ").split()
key = f"c{a} xor c{b}" 
crib_xor(crib, key)

#XOR plaintext words with original ciphertexts to recover key fragments 
def plaintext_xor(plaintext, ciphertext):
    key_res = []
    for i in range(0, len(ciphertext), len(plaintext)):
        chunk = ciphertext[i:i+len(plaintext)]
        if len(chunk) == len(crib):
            res = xor_strings(plaintext, chunk)
            key_res.append(res.hex())
            print(f"At index {i}: {res.hex()}")
    return(key_res)

#allow manual input of picking ciphertext to XOR
#XORplaintext = input("Please enter plaintext string: ").encode()
#XORciphertextindex = int(input("PLease enter the numer of the ciphertext to XOR (1-10): "))
#XORciphertext = encoded_ciphertests[XORciphertextindex]
#keyfrag = plaintext_xor(XORplaintext,XORciphertext)

#function to do frequency analysis on xored strings
def bigram_counter(input_string):
    res = Counter(input_string[idx : idx + 2] for idx in range(len(input_string)-1))
    return(res)

for i in range(len(xoredciphertexts_hex_list)):
    res = bigram_counter(xoredciphertexts_hex_list[i])
    frequencycounts.append(res)
    #print(res)

Please enter a crib word:  the
Please enter the numbers of the two ciphertexts to XOR separated by a space:  1 3


At position 0: b'why'
At position 3: b'&z$'
At position 6: b'f:t'
At position 9: b'<<u'
At position 12: b':yk'
At position 15: b'$DB'
At position 18: b'the'
At position 21: b'2qc'
At position 24: b'rye'
At position 27: b'1 c'
At position 30: b'i8x'
At position 33: b'hl)'
At position 36: b'j}r'
At position 39: b'|3k'
At position 42: b'\x1cY-'
At position 45: b'zxc'
At position 48: b'={h'
At position 51: b'u<q'
At position 54: b'v`y'
At position 57: b'&o~'
At position 60: b'`ei'
At position 63: b'1n,'
At position 66: b'nar'
At position 69: b'-.m'
At position 72: b'hi2'
At position 75: b'alf'
